## HIR Charts

In [2]:
import pandas as pd
import numpy as np

In [2]:
nat_hir = pd.read_csv('natl_home_inc_hir_8918.csv')  # Zillow (summary) + Census annual inc
ca_hir = pd.read_csv('ca_state_home_inc_hir_8918.csv')  # Zillow (summary) + Census annual inc
#ca_counties_hir = pd.read_csv('ca_counties_home_inc_hir_8918.csv')  # Zillow (indiv. CA counties) + SAIPE inc
reg_hir = pd.read_csv('reg_hirs_chart_8918.csv')  # FHFA home + SAIPE inc (reg_hirs_8918.py)
county_hir = pd.read_csv('county_home_inc_hir_8918.csv')  # FHFA home + SAIPE inc
county_units = pd.read_csv('county_housing_units_8918.csv')  # Census unit counts
#zillow_county = pd.read_csv('zillow_county_home_inc_hir_8918.csv')
hhcount = pd.read_csv('hh_count_hir_8918.csv')

In [3]:
%run chart_theme.py

---

### National Median HIR

In [4]:
natl_med = alt.Chart(nat_hir, title='National Median Home-to-Income Ratio (HIR)'
                    ).mark_line(size=7, strokeCap='round', color='#89c3ff',
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(ticks=False, values=list(range(1990, 2019, 4)))), 
                                        y=alt.Y('HIR', title='*Median home price / median annual income', scale=alt.Scale(domain=[1,4.5]), axis=alt.Axis(values=list(np.arange(1.5,5,.5)), titleY=-40)))
natl_med

alt.Chart(...)

In [5]:
natl_med.save('natl_med_hir.png', scale_factor=3)

---

### HIR Distribution (FHFA county data)

In [6]:
#plot_df = zillow_county.copy()
plot_df = county_hir.copy()
plot_df['pct_bin'] = plot_df.HIR.round(1)

In [7]:
plot_df.head(2)

,Year,State,FIPS,Areaname,Med_Home,Med_Inc,HIR,HIR_Chg,pct_bin
0,1989,AL,1001,"Autauga, AL",86878,29113,2.98,NaN,3.0
1,1990,AL,1001,"Autauga, AL",86539,30118,2.87,-0.036913,2.9


In [608]:
#fhfa_calc_nat_med = pd.read_csv('natl_home_inc_hir_OLD.csv')

In [609]:
#hir_diff_df = pd.DataFrame({'Year':range(1989, 2019), 'New_HIR':nat_hir.HIR.values, 'Orig_HIR':fhfa_calc_nat_med.HIR.values})
#hir_diff_df['HIR_Diff'] = hir_diff_df.New_HIR - hir_diff_df.Orig_HIR
#hir_diff_df['HIR_Adj_Mult'] = 1 + hir_diff_df.HIR_Diff / hir_diff_df.Orig_HIR

#### Adjusting bin values to be consistent w/ national annual median:

In [610]:
"""adj_dict = {}
for year in range(1989, 2019):
    adj_dict[year] = hir_diff_df.HIR_Adj_Mult[hir_diff_df.Year == year].iloc[0]"""

In [611]:
#plot_df['Adj_Mult'] = plot_df.Year.map(adj_dict)
#plot_df['pct_bin'] = round(plot_df.HIR * plot_df.Adj_Mult, 1)

In [8]:
plot_df.head(2)

,Year,State,FIPS,Areaname,Med_Home,Med_Inc,HIR,HIR_Chg,pct_bin
0,1989,AL,1001,"Autauga, AL",86878,29113,2.98,NaN,3.0
1,1990,AL,1001,"Autauga, AL",86539,30118,2.87,-0.036913,2.9


In [11]:
# Dict contains household count for each HIR bin
yrs_dict = {}
for year in plot_df.Year.unique():
    yr_dict = {}    
    yr_df = plot_df[plot_df.Year == year]
    vals_dict = {}
    meds_dict = {}
    for val in yr_df.pct_bin.unique():
        bin_fips = yr_df.FIPS[yr_df.pct_bin == val].values
        vals_dict[val] = county_units[str(year)][county_units.FIPS.isin(bin_fips)].sum() / 1000000  # Dividing by 1 million for simpler chart labels
    yr_dict['bins'] = vals_dict
    yr_dict['med'] = nat_hir.HIR[nat_hir.Year == year].iloc[0].round(1)
    yrs_dict[year] = yr_dict

In [12]:
# Identifying max to manipulate chart so y-axis is consistent for all charts
for year in yrs_dict:
    print(year, max(yrs_dict[year]['bins'].values()))

1989 7.645715
1990 8.037839
1991 8.147636
1992 7.392059
1993 8.077823
1994 7.720695
1995 7.948949
1996 8.331803
1997 9.166335
1998 10.614931
1999 11.498596
2000 12.00793
2001 8.302649
2002 8.516003
2003 7.421635
2004 5.277334
2005 5.340812
2006 6.143999
2007 5.114154
2008 6.137891
2009 5.832728
2010 7.117973
2011 7.465723
2012 8.215467
2013 8.695751
2014 7.928956
2015 8.916415
2016 8.247151
2017 7.990156
2018 7.313988


In [13]:
# Identifying max bin to manipulate chart so x-axis is consistent for all charts (ended up excluding values >= 13)
for year in yrs_dict:
    print(year, max(yrs_dict[year]['bins'].keys()))

1989 14.3
1990 12.8
1991 11.6
1992 10.4
1993 9.4
1994 8.3
1995 8.2
1996 7.9
1997 7.8
1998 8.9
1999 9.2
2000 10.6
2001 11.4
2002 11.7
2003 13.0
2004 13.5
2005 15.6
2006 17.6
2007 17.8
2008 15.4
2009 15.0
2010 13.7
2011 13.0
2012 13.0
2013 13.0
2014 12.4
2015 13.1
2016 15.2
2017 13.6
2018 13.4


#### 1989-2018 Distribution (annual charts for GIF)
*No y-axis lines or labels

In [14]:
x2 = np.arange(1.2,13,.4)
counter = 0

for year in yrs_dict:
    counts_df = pd.DataFrame(yrs_dict[year]['bins'].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    counts_df = counts_df[counts_df.Bins < 13]  # Removing outliers to maintain consistent axes dimensions
    line_df = pd.DataFrame({'x':[1.2], 'x2':[x2[counter]], 'y':[12.5]})
    text_df = pd.DataFrame({'Bins':[1,13], 'Counts':[12,12], 'Text':[1989, 2018]})
    
    # Charts
    chart = alt.Chart(counts_df, title='HIR Distribution (1989-2018)'
                     ).mark_bar(size=3.5, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                     ).encode(x=alt.X('Bins', title='Home-to-Income Ratio', scale=alt.Scale(domain=[1,13]), axis=alt.Axis(values=list(range(1,14)))), 
                              y=alt.Y('Counts:Q', axis=None, title='Households (millions)', scale=alt.Scale(domain=[0,13])),
                              color=alt.condition(alt.datum.Bins == yrs_dict[year]['med'], 
                                                  alt.value('#89c3ff'), 
                                                  alt.value('#2d6e9f')))
    line = alt.Chart(line_df).mark_rule(color='lightgray', size=20).encode(x=alt.X('x'), x2=alt.Y2('x2'), y=alt.Y('y', axis=None))
    text1 = alt.Chart(text_df.query('Bins == 1')).mark_text(dx=-20, dy=-14, fontSize=20).encode(x=alt.X('Bins', axis=None), y=alt.Y('Counts', axis=None), text=alt.Text('Text'))
    text2 = alt.Chart(text_df.query('Bins == 13')).mark_text(dx=20, dy=-14, fontSize=20).encode(x=alt.X('Bins', axis=None), y=alt.Y('Counts', axis=None), text=alt.Text('Text'))

    plot = chart + line + text1 + text2
    plot.save('HIR_dist_no_labels' + str(year) + '.png', scale_factor=3)
    counter += 1

1

---

#### 2018 Distribution
*w/ y-axis lines & labels

In [115]:
df = pd.DataFrame(yrs_dict[2018]['bins'].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
df = df[df.Bins < 13]  # Removing outliers to maintain consistent axes dimensions
text_line_df = pd.DataFrame({'x1':[3.1], 'x2':[10], 'x3':[11], 'x4':[12], 'x5':[11.5], 'y1':[7.3], 'y2':[7.3], 'y3':[8], 'y4':[7.9], 'y5':[8.4], 'Text1':['7.3 million households'], 'Text2':['had a HIR of 2.9']})

chart = alt.Chart(df, title='National HIR Distribution (2018)'
                 ).mark_bar(size=3.5, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                 ).encode(x=alt.X('Bins', title='Home-to-Income Ratio', scale=alt.Scale(domain=[1,13]), axis=alt.Axis(values=list(range(1,14)))), 
                          y=alt.Y('Counts:Q', title='Households (millions)', scale=alt.Scale(domain=[0,10]), axis=alt.Axis(values=list(range(2,12,2)))),
                          color=alt.condition(alt.datum.Bins == yrs_dict[year]['med'], 
                                              alt.value('#89c3ff'), 
                                              alt.value('#2d6e9f')))
line1 = alt.Chart(text_line_df).mark_rule(size=2, color='#999999', strokeDash=[1,3]).encode(x='x1', x2='x2', y='y1', y2='y2')
line2 = alt.Chart(text_line_df).mark_rule(size=2, color='#999999', strokeDash=[1,3]).encode(x='x2', x2='x3', y='y2', y2='y3')
text1 = alt.Chart(text_line_df.query('x1==3.1')).mark_text(dx=-20, dy=-14, fontSize=16).encode(x=alt.X('x4', axis=None), y=alt.Y('y5', axis=None), text=alt.Text('Text1'))
text2 = alt.Chart(text_line_df.query('x1==3.1')).mark_text(dx=-20, dy=-14, fontSize=16).encode(x=alt.X('x5', axis=None), y=alt.Y('y4', axis=None), text=alt.Text('Text2'))

dist18 = chart + line1 + line2 + text1 + text2
dist18

alt.LayerChart(...)

In [116]:
dist18.save('HIR_dist_18.png', scale_factor=3)

---

#### 1989-1999 Distribution

In [101]:
years = range(1989,2000)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year]['bins'].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 13]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,14,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='HIR Distribution - \'89-\'99'
                   ).mark_bar(color='#2d6e9f', size=4.2, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home-to-Income Ratio', axis=alt.Axis(values=list(range(1,14)), titleFontSize=18, labelFontSize=18, titleY=35)), 
                                      y=alt.Y('Counts:Q', title='Households (millions)', scale=alt.Scale(domain=[0,12]), axis=alt.Axis(titleFontSize=18, labelFontSize=18, values=list(range(2,14,2)))))
    variables.append(var)
    counter += 1

combined8999 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart (b/c it's already assigned to 'test')
    combined8999 += var

combined8999

alt.LayerChart(...)

In [15]:
combined8999.save('combined8999.png', scale_factor=3)

---

#### 2000-2010 Distribution

In [112]:
years = range(2000,2010)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year]['bins'].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 13]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,14,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='HIR Distribution - \'00-\'09'
                   ).mark_bar(color='#2d6e9f', size=4.2, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home-to-Income Ratio', axis=alt.Axis(values=list(range(1,14)), titleFontSize=18, labelFontSize=18, titleY=35)), 
                                      y=alt.Y('Counts:Q', title='Households (millions)', scale=alt.Scale(domain=[0,12]), axis=alt.Axis(titleFontSize=18, labelFontSize=18, values=list(range(2,14,2)))))
    variables.append(var)
    counter += 1

combined0009 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart
    combined0009 += var

combined0009

alt.LayerChart(...)

In [17]:
combined0009.save('combined0009.png', scale_factor=3)

---

#### 2010-2018 Distribution

In [103]:
years = range(2010,2019)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year]['bins'].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 13]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,14,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='HIR Distribution - \'10-\'18'
                   ).mark_bar(color='#2d6e9f', size=4.2, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home-to-Income Ratio', axis=alt.Axis(values=list(range(1,14)), titleFontSize=18, labelFontSize=18, titleY=35)), 
                                      y=alt.Y('Counts:Q', title='Households (millions)', scale=alt.Scale(domain=[0,12]), axis=alt.Axis(titleFontSize=18, labelFontSize=18, values=list(range(2,14,2)))))
    variables.append(var)
    counter += 1

combined1018 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart
    combined1018 += var

combined1018

alt.LayerChart(...)

In [20]:
combined1018.save('combined1018.png', scale_factor=3)

---

In [113]:
x = (combined8999.properties(height=400, width=600) | combined0009.properties(height=400, width=600)) & combined1018.properties(height=400, width=600)
x.configure_title(fontSize=24)

alt.VConcatChart(...)

In [114]:
x.save('all_dist_charts_8918.png', scale_factor=3)

---

### HIR by Region:

In [57]:
region_text_df = pd.DataFrame({'Year':(range(1989, 2019)), 'Numbers':(list(range(30))), 'Text':(['West', 'Northeast', 'South', 'Midwest'] + list(np.full(26, np.nan)))})

In [98]:
reg_chart = alt.Chart(reg_hir, title='Median HIR by Region').mark_line(size=5, strokeCap='round').encode(
    x=alt.X('Year:O', axis=alt.Axis(values=list(range(1990,2019,4)), titleFontSize=18, labelFontSize=18)), 
    y=alt.Y('HIR', axis=alt.Axis(titleFontSize=18, labelFontSize=18, values=list(range(2,8)), format='a'), scale=alt.Scale(domain=[1,7.5])),
    color=alt.Color('Region', legend=None, scale=alt.Scale(domain=['West', 'Northeast', 'South', 'Midwest'], range=['#004d66', '#00a58a', '#ffd5b0', '#c86300'])))

west = alt.Chart(region_text_df.query('Numbers == 0')
                   ).mark_text(dx=25, dy=-340, fontSize=20, fontWeight='normal', color='#004d66'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
northeast = alt.Chart(region_text_df.query('Numbers == 1')
                   ).mark_text(dx=45, dy=-208, fontSize=20, fontWeight='normal', color='#00a58a'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
south = alt.Chart(region_text_df.query('Numbers == 2')
                   ).mark_text(dx=26, dy=-88, fontSize=20, fontWeight='normal', color='#ffbe94'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
midwest = alt.Chart(region_text_df.query('Numbers == 3')
                   ).mark_text(dx=37, dy=12, fontSize=20, fontWeight='normal', color='#c86300'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))

region_chart = reg_chart + west + northeast + south + midwest
region_chart.configure_title(fontSize=24)

alt.LayerChart(...)

In [99]:
region_chart.save('region_hir_8918.png', scale_factor=3)

---

### HIR by County Household Count:

In [159]:
hir_pop = alt.Chart(hhcount, title='HIR by County Household Count').mark_line(size=5, strokeCap='round').encode(
x=alt.X('Year:O', axis=alt.Axis(values=list(range(1990, 2019,4)), labelFontSize=18)),
y=alt.Y('HIR', title='Median HIR for county groupings by number of households', scale=alt.Scale(domain=[0,8]), axis=alt.Axis(titleFontSize=18, labelFontSize=18, values=list(range(1,9)), format='a')),
color=alt.Color('HH_Count', scale=alt.Scale(domain=list(hhcount.HH_Count.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)')
).configure_legend(orient='none', legendX=610, legendY=95, titleFont='Lato', titleFontSize=20, titleLimit=250, labelFont='Lato', labelFontSize=20, symbolSize=200, symbolStrokeWidth=5)

hir_pop.configure_title(fontSize=24)

alt.Chart(...)

In [160]:
hir_pop.save('hir_hh_count.png', scale_factor=3)

---

### LA vs. Other HIR 

In [63]:
# Ranking CA Counties by Avg. HIR
ca_counties = county_hir[(county_hir.FIPS > 6000) & (county_hir.FIPS < 7000)]
fips_dict = {}
for fips in ca_counties.FIPS.unique():
    df = ca_counties[ca_counties.FIPS == fips]
    fips_dict[fips] = df.HIR.mean().round(3)
    
avg_hirs = sorted(fips_dict.items(), key=lambda x:x[1], reverse=True)

In [64]:
avg_hirs[:8]  # LA has the 6th highest avg. HIR in CA

[(6075, 9.089),
 (6087, 8.465),
 (6041, 8.078),
 (6081, 7.745),
 (6083, 7.408),
 (6037, 7.283),
 (6079, 7.277),
 (6045, 7.181)]

In [65]:
la_df = county_hir[county_hir.FIPS == 6037]

In [66]:
ca_df = county_hir[(county_hir.FIPS != 6037) & (county_hir.FIPS > 6000) & (county_hir.FIPS < 7000)]

In [67]:
text_df = la_df.copy()
text_df['Text'] = list(np.full(len(text_df)-5, np.nan)) + ['CA', 'Counties', 'National', 'California', 'LA County']

In [78]:
ca_counties = alt.Chart(ca_df, title='LA vs. Other Regions').mark_line().encode(
    x=alt.X('Year:O', axis=alt.Axis(values=list(range(1990, 2019,4)), labelFontSize=18)), 
    y=alt.Y('HIR', title='LA county median HIR relative to CA median, nat\'l median and CA\'s other counties', scale=alt.Scale(domain=[0,14]), axis=alt.Axis(values=list(range(2,16,2)), titleFontSize=18, labelFontSize=18)),
    color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))

nat_med = alt.Chart(nat_hir).mark_line(color='#9a9a9a', size=5, strokeCap='round').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

ca_med = alt.Chart(ca_hir).mark_line(size=5, strokeCap='round').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

la = alt.Chart(la_df).mark_line(color='#fbcb42', size=5, strokeCap='round').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

CA_text = (
    alt.Chart(text_df.query('Year == 2014'))
    .mark_text(dx=103, dy=60, fontSize=20, fontWeight='normal', color='#afafaf')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

cou_text = (
    alt.Chart(text_df.query('Year == 2015'))
    .mark_text(dx=109, dy=83, fontSize=20, fontWeight='normal', color='#afafaf')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)
nat_text = (
    alt.Chart(text_df.query('Year == 2016'))
    .mark_text(dx=94, dy=141, fontSize=22, fontWeight='bold', color='#858585')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

ca_text = (
    alt.Chart(text_df.query('Year == 2017'))
    .mark_text(dx=80, dy=28, fontSize=22, fontWeight='bold', color='#15607a')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

la_text = (
    alt.Chart(text_df.query('Year == 2018'))
    .mark_text(dx=65, dy=-3, fontSize=22, fontWeight='bold', color='#ffca00')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

chart = ca_counties + nat_med + ca_med + la + CA_text + cou_text + nat_text + ca_text + la_text
chart.configure_title(fontSize=24)

alt.LayerChart(...)

In [79]:
chart.save('la_ca_nat_counties_hir.png', scale_factor=3)

---

## Fed Funds Rate

In [4]:
fed = pd.read_csv('fed_int_rate_9020.csv')

In [9]:
fed_plot = alt.Chart(fed, title='Target Fed Funds Rate').mark_line(size=2.5).encode(x=alt.X('Date:T', axis=alt.Axis(labelFontSize=18)), 
                                                                         y=alt.Y('Target_Rate', 
                                                                                 title='Benchmark interest rate set by the Federal Reserve', 
                                                                                 axis=alt.Axis(values=list(np.arange(.01,.09,.01)), titleX=-30, titleY=-30, titleFontSize=18, labelFontSize=18, labelPadding=30, format='p'))).properties(width=600)
fed_plot

alt.Chart(...)

In [10]:
fed_plot.save('fed_funds_rate.png', scale_factor=3)

---

## Rent Share of Income

In [57]:
states_rent = pd.read_csv('states_rent_inc_hir_1018.csv')
states_rent.Rent_Per_Inc = states_rent.Rent_Per_Inc / 100

In [58]:
states_rent.head(2)

,Year,State,Income,Med_Rent,Rent_Per_Inc
0,2010,Alabama,3411,860.0,0.25
1,2011,Alabama,3549,860.0,0.24


In [112]:
ca_counties = alt.Chart(states_rent, title='Rent % of Income').mark_line().encode(
    x=alt.X('Year:O'), 
    y=alt.Y('Rent_Per_Inc', title='CA median HIR relative to other states', scale=alt.Scale(domain=[.1,.45]), axis=alt.Axis(values=list(np.arange(.05,.5,.05)), format='p')),
    color=alt.Color('State', scale=alt.Scale(range=['#e0e0e5']), legend=None))
ca_counties

alt.Chart(...)

In [63]:
list(range(.05,.5))

TypeError: 'float' object cannot be interpreted as an integer

---

### Counties where over 30% of income is devoted to Housing or Rent

In [222]:
over30 = pd.read_csv('county_housing_over_30.csv')

In [223]:
over30.head(2)

,Year,FIPS,County,Owners30,Renters30
0,2010,1003,"Baldwin County, Alabama",26.9,50.1
1,2010,1015,"Calhoun County, Alabama",29.7,45.8


In [225]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [234]:
alt.Chart(over30).mark_line().encode(x='Year:O', y=alt.Y('Owners30'), color=alt.Color('County', legend=None, scale=alt.Scale(range=['#e0e0e5'])))

alt.Chart(...)